# Experiment Notebook - New Test
This notebook will illustrate how to create and run a new experiment, or Test Run.

In [ ]:
import sqlite3
# Create a connection to the database
db_connection = sqlite3.connect('experiment.db')


## 5. Creating a new Test Run
Each experiment is represented by a Test Run, which will consume a Dataset to perform completions on a related set of Questions.

The generated Responses and their Contexts will then be evaluated by the configured Test Eval functions.

In [ ]:
# Create a new Test Run
from packages.data.src.eval_data.models.testrun import TestRunModel, TestRunType

test_run_model = TestRunModel(db_connection)
test_run = test_run_model.add_or_get_test_run(TestRunType(datasource_id=3, description="Test Run 4"))
print(f"Test Run added with ID: {test_run.id}")


## 6. Load document text index


In [ ]:
from packages.data.src.eval_data.models.document import DocumentModel
from typing import List

documents = DocumentModel(db_connection).get_documents_by_datasource(datasource_id=test_run.datasource_id)

# Get questions and answers
from packages.data.src.eval_data.models.question import QuestionModel, QuestionType
from packages.data.src.eval_data.models.response import ResponseModel, ResponseType
from packages.data.src.eval_data.models.context import ContextModel, ContextType

questions: List[QuestionType] = []
for doc in documents:
    questions.extend(QuestionModel(db_connection).get_questions_by_document_id(document_id=doc.id))

# Build the document index
PERSIST_PATH = f"datasets/query_engine/{test_run.datasource_id}"

from typing import List
import os
from documents import load_documents


nodes = load_documents(documents)



## 7. Create instance of the LLM query engine

In [ ]:
from packages.scripts.src.eval_scripts.utils import build_query_engine

query_engine = build_query_engine(nodes, PERSIST_PATH)

## 8. Response Generation
Utilizing the RAG system to generate responses to the questions during test runs.

In [ ]:
# Generate responses
for q in questions:
    res = query_engine.query(q.question)
    response = ResponseType(
        test_run_id = test_run.id,
        question_id = q.id,
        response = res.response
    )
    response_id = ResponseModel(db_connection).add_response(response)
    print(f"Response created: {response_id} - len: {len(response.response)}")

    context_model = ContextModel(db_connection)
    for c in res.source_nodes:
        context_model.add_context(
            ContextType(
                response_id=response_id, 
                text=c.node.get_content(),
                similarity_score=c.score
            )
        )